In [ ]:
!pip install tensorflow opencv-python pandas openpyxl
!pip install deepface mtcnn opencv-python pandas pytz
!pip install lz4

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.2/87.2 kB 6.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.6/108.6 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 52.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.0/85.0 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 77.9 MB/s eta 0:00:00
  Created wheel for fire: filename=fire-0.7.0-py3-none-any.whl size=114249 sha256=33865bf9e091ecac63042e28bf80b13ce89810e28656f9036120f8eebad22403
  Stored in directory: /root/.cache/pip/wheels/46/54/24/1624fd5b8674eb1188623f7e8e17cdf7c0f6c24b609dfb8a89
Successfully built fire


In [ ]:
import joblib

# Load the trained model
model_path = "/content/face_recognition_model.pkl"  # Ensure this path is correct
model = joblib.load(model_path)

print("Model loaded successfully!")


Model loaded successfully!


In [ ]:
import pickle
import pandas as pd
from datetime import datetime
import pytz
from deepface import DeepFace
from mtcnn import MTCNN
import cv2
import numpy as np
from IPython.display import display, Javascript
from google.colab.output import eval_js
from base64 import b64decode

# Load the trained classifier
model_path = "/content/face_recognition_model.pkl"
with open(model_path, "rb") as f:
    model = pickle.load(f)

# Load face detector
detector = MTCNN()

# Set local timezone to IST
LOCAL_TIMEZONE = "Asia/Kolkata"

def mark_attendance(student_name):
    filename = "attendance.csv"
    try:
        df = pd.read_csv(filename)
    except FileNotFoundError:
        df = pd.DataFrame(columns=["Name", "Date", "Time"])

    now = datetime.now(pytz.timezone(LOCAL_TIMEZONE))
    date = now.strftime("%Y-%m-%d")
    time = now.strftime("%H:%M:%S")

    if not ((df["Name"] == student_name) & (df["Date"] == date)).any():
        new_entry = pd.DataFrame([[student_name, date, time]], columns=["Name", "Date", "Time"])
        df = pd.concat([df, new_entry], ignore_index=True)
        df.to_csv(filename, index=False)
        print(f"✅ Attendance Marked for {student_name} at {time}")
    else:
        print(f"ℹ️ Attendance already marked for {student_name} today.")

def take_photo():
    js = Javascript('''
        async function captureImage() {
            const video = document.createElement('video');
            const canvas = document.createElement('canvas');
            const div = document.createElement('div');
            const capture = document.createElement('button');
            const stop = document.createElement('button');

            capture.textContent = '📸 Take Photo';
            stop.textContent = '❌ End';
            div.appendChild(capture);
            div.appendChild(stop);
            document.body.appendChild(div);
            document.body.appendChild(video);

            const stream = await navigator.mediaDevices.getUserMedia({ video: true });
            video.srcObject = stream;
            await video.play();

            return await new Promise((resolve) => {
                capture.onclick = () => {
                    canvas.width = video.videoWidth;
                    canvas.height = video.videoHeight;
                    canvas.getContext('2d').drawImage(video, 0, 0);
                    stream.getTracks().forEach(track => track.stop());
                    video.remove();
                    div.remove();
                    resolve(canvas.toDataURL('image/jpeg'));
                };
                stop.onclick = () => {
                    stream.getTracks().forEach(track => track.stop());
                    video.remove();
                    div.remove();
                    resolve('STOP');
                };
            });
        }
        captureImage();
    ''')

    display(js)
    data = eval_js("captureImage()")
    if data == 'STOP':
        return None

    binary = b64decode(data.split(',')[1])
    filename = "photo.jpg"
    with open(filename, 'wb') as f:
        f.write(binary)

    return filename

# Start the capture & prediction loop
while True:
    image_path = take_photo()
    if image_path is None:
        print("❌ Attendance session ended.")
        break

    print(f"✅ Captured: {image_path}")
    img = cv2.imread(image_path)
    rgb_img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    faces = detector.detect_faces(rgb_img)
    if not faces:
        print("❌ No face detected. Try again.")
        continue

    for face in faces:
        x, y, w, h = face['box']
        x, y = abs(x), abs(y)
        face_crop = rgb_img[y:y+h, x:x+w]
        face_crop = cv2.resize(face_crop, (160, 160))

        try:
            embedding = DeepFace.represent(face_crop, model_name="Facenet", enforce_detection=False)
            face_embedding = np.array(embedding[0]["embedding"]).reshape(1, -1)
            name = model.predict(face_embedding)[0]
            mark_attendance(name)
            print(f"✅ Recognized: {name}")
        except Exception as e:
            print(f"❌ Face embedding/prediction failed: {str(e)}")


<IPython.core.display.Javascript object>

✅ Captured: photo.jpg
ℹ️ Attendance already marked for Rohinth today.
✅ Recognized: Rohinth


<IPython.core.display.Javascript object>